In [1]:
import pandas as pd
import workdays
import datetime
from datetime import datetime, timedelta, date
from xbbg import blp
import pdblp
# https://data.bloomberglp.com/labs/sites/2/2013/12/blpapi-developers-guide-1.38.pdf
# https://data.bloomberglp.com/professional/sites/10/2017/03/BLPAPI-Core-Developer-Guide.pdf

In [2]:
FX0 = ['USD', 'GBP', 'HKD', 'CAD', 'AUD', 'EUR','CHF']
FX = [x + 'JPY BGN Curncy' for x in FX0 ]
FX = blp.bdp(tickers=FX, flds='px_last')
FX.index = [x[:3] for x in FX.index.tolist()]
FX

,px_last
AUD,97.7750
CAD,107.1110
CHF,171.0680
EUR,162.0300
GBP,190.8190
HKD,18.6796
USD,145.6500


In [3]:
def BDP(L,YK):
    T = [L[i] + 'A ' + YK[i] for i in range(len(L))]
    T1 = [L[i] + '1 ' + YK[i] for i in range(len(L))]

    df = blp.bdp(tickers=T, flds=['name','currency','exch_code','undl_spot_ticker']).loc[T,:]
    tmp = [x + ' Index' for x in df['undl_spot_ticker'].drop_duplicates().tolist()]
    tmp = blp.bdp(tickers=tmp, flds='country_iso')
    tmp.index = [item.replace(' Index', '') for item in tmp.index.tolist()]

    df.loc[:, 'risk'] = df['undl_spot_ticker'].map(tmp['country_iso'])
    df.insert(0,'ticker', L)
    df = df.reset_index(drop=True)

    df1 = blp.bdp(tickers=T1, flds=['contract_value','volume_avg_20d','volatility_20d']).loc[T1,:]
    df1.columns = ['value/unit','volume','vola']
    df = pd.concat([df, df1.reset_index(drop=True)], axis=1)
    df = df.rename(columns={'currency':'crncy','exch_code':'exch','undl_spot_ticker':'undl','contract_value':'value/unit'})
    
    tmp = []
    for i in range(len(L)):
        tmp  += [(blp.bdh(T1[i], 'px_last', '19991201', '20220101', Per='Q' ).reset_index().iloc[0,0]).strftime('%Y%m')]
    df['initinal_t/d'] = tmp
    df = df.replace('199912', '-')

    df['fx'] = 1.0
    df.loc[df['crncy'] != 'JPY', 'fx'] = df['crncy'].map(FX['px_last'])
    df['value/unit_JPY'] = (df['value/unit'] * df['fx']).round(-3)
    df = df.drop('fx', axis=1)

    tmp = df['risk']
    df = df.drop('risk', axis=1)
    df.insert(3, 'risk', tmp)

    # df['value/unit'] = df.pop('value/unit')
    df = df.drop('value/unit', axis=1)
    df['value/unit_JPY'] = df.pop('value/unit_JPY')
    df['initinal_t/d'] = df.pop('initinal_t/d')


    df['vola'] = df['vola'].round()
    df['volume'] = df['volume'].fillna(0)
    
    df['illiquidity'] = 0
    df['mini'] = '-'
    df['size_mini'] = '-'
    df['illiquidity_mini'] = 0
    
    for i in range(len(df)):
        
        if df.loc[i,'volume'] < 3000:
            df.loc[i,'illiquidity'] = 1
        
        if df.loc[:i,'undl'].tolist().count(df.loc[i,'undl']) > 1:
            df.loc[i,'mini'] = 1
            
            r = df.loc[df['undl'] == df.loc[i,'undl'],'size_mini'].index[0]
            df.loc[r,'mini'] = df.loc[i,'ticker']
            df.loc[r,'size_mini'] = (df.loc[i,'value/unit_JPY'] / df.loc[r,'value/unit_JPY']).round(3)
            
            if df.loc[i,'illiquidity'] == 1:
                df.loc[r,'illiquidity_mini'] = 1
            
    for j in ['value/unit_JPY','volume']:
        df[j] = df[j].apply('{:,.0f}'.format)
        
    
    return df
    
L= ['ES','NQ','DM','RTY','VG','CF','GX','EO','IB','SW','Z ','SM','PT','XP','TP','HI','XU','JGS','TWT']
L += ['HWA','HWB','HWI','HWR','DFW','CMS','MFO','ID','MPT','MXP','TMI','HU','MES']

# L= ['ES','VG','PT']
# L += ['HWA','HWB','MPT']
YK = ['Index'] * len(L)

df = BDP(L,YK)
print('Equity Futures')
print(df.shape)
display(df[df['mini'] != 1])

print('mini(smaller size)')
display(df[df['mini'] == 1])
print('illiquidity')
df[df['illiquidity'] == 1]

Equity Futures
(32, 14)


,ticker,name,crncy,risk,exch,undl,volume,vola,value/unit_JPY,initinal_t/d,illiquidity,mini,size_mini,illiquidity_mini
0,ES,S&P500 EMINI FUT Sep24,USD,US,CME,SPX,"1,800,057",21.0,"40,837,000",-,0,HWA,0.1,0
1,NQ,NASDAQ 100 E-MINI Sep24,USD,US,CME,NDX,"630,840",30.0,"57,233,000",-,0,HWB,0.1,0
2,DM,DJIA MINI e-CBOT Sep24,USD,US,CBT,INDU,"142,104",17.0,"29,785,000",200206,0,HWI,0.2,0
3,RTY,E-Mini Russ 2000 Sep24,USD,US,CME,RTY,"212,656",29.0,"15,772,000",201709,0,HWR,0.1,0
4,VG,EURO STOXX 50 Sep24,EUR,EU,EUX,SX5E,"735,750",19.0,"7,925,000",-,0,-,-,0
5,CF,CAC40 10 EURO FUT Sep24,EUR,FR,EOP,CAC,"56,194",16.0,"12,229,000",-,0,CMS,0.1,1
6,GX,DAX INDEX FUTURE Sep24,EUR,DE,EUX,DAX,"41,418",18.0,"75,060,000",-,0,DFW,0.2,0
7,EO,AMSTERDAM IDX FUT Sep24,EUR,NL,EOE,AEX,"11,631",20.0,"29,511,000",-,0,MFO,0.1,1
8,IB,IBEX 35 INDX FUTR Sep24,EUR,ES,MFM,IBEX,"12,840",20.0,"18,135,000",-,0,ID,0.1,1
9,SW,MINI FTSE/MIB FUT Sep24,EUR,IT,MIL,FTSEMIB,"8,067",22.0,"5,414,000",200403,0,-,-,0


mini(smaller size)


,ticker,name,crncy,risk,exch,undl,volume,vola,value/unit_JPY,initinal_t/d,illiquidity,mini,size_mini,illiquidity_mini
19,HWA,SP500 MIC EMIN FUTSep24,USD,US,CME,SPX,"1,182,828",21.0,"4,084,000",201906,0,1,-,0
20,HWB,NASD100 MICRO EMINSep24,USD,US,CME,NDX,"1,728,954",30.0,"5,723,000",201906,0,1,-,0
21,HWI,DJIA MICR MIN CBOTSep24,USD,US,CBT,INDU,"126,710",17.0,"5,957,000",201906,0,1,-,0
22,HWR,MICRO EMIN RUS2000Sep24,USD,US,CME,RTY,"112,849",29.0,"1,577,000",201906,0,1,-,0
23,DFW,DAX Mini Future Sep24,EUR,DE,EUX,DAX,"28,549",18.0,"15,010,000",201512,0,1,-,0
24,CMS,CAC40 Mini Future Sep24,EUR,FR,EOP,CAC,16,16.0,"1,223,000",201309,1,1,-,0
25,MFO,AMSTERDAM MIN ID Sep24,EUR,NL,EOE,AEX,7,20.0,"2,951,000",201309,1,1,-,0
26,ID,IBEX MINI IDX FUT Sep24,EUR,ES,MFM,IBEX,"2,902",20.0,"1,813,000",200112,1,1,-,0
27,MPT,MINI S&P/TSX 60 Sep24,CAD,CA,MSE,SPTSX60,20,16.0,"7,401,000",201106,1,1,-,0
28,MXP,Mini SPI 200 FUT Sep24,AUD,AU,SFE,AS51,0,20.0,"3,907,000",201512,1,1,-,0


illiquidity


,ticker,name,crncy,risk,exch,undl,volume,vola,value/unit_JPY,initinal_t/d,illiquidity,mini,size_mini,illiquidity_mini
24,CMS,CAC40 Mini Future Sep24,EUR,FR,EOP,CAC,16,16.0,"1,223,000",201309,1,1,-,0
25,MFO,AMSTERDAM MIN ID Sep24,EUR,NL,EOE,AEX,7,20.0,"2,951,000",201309,1,1,-,0
26,ID,IBEX MINI IDX FUT Sep24,EUR,ES,MFM,IBEX,"2,902",20.0,"1,813,000",200112,1,1,-,0
27,MPT,MINI S&P/TSX 60 Sep24,CAD,CA,MSE,SPTSX60,20,16.0,"7,401,000",201106,1,1,-,0
28,MXP,Mini SPI 200 FUT Sep24,AUD,AU,SFE,AS51,0,20.0,"3,907,000",201512,1,1,-,0


In [4]:
df1 = df.loc[df['mini']!=1,['ticker','name','crncy','risk','value/unit_JPY','mini','size_mini','illiquidity_mini']].copy()
df1.loc[df1['illiquidity_mini']==1,'mini'] = '(illiquidity)'
df1.loc[df1['illiquidity_mini']==1,'size_mini'] = '-'
df1 = df1.drop('illiquidity_mini', axis=1)

for i in [200,300,500,1000]:
# for i in [300]:    
    df1[str(i) + ' mil'] = df1['value/unit_JPY'].str.replace(',', '').astype(float) * df1['size_mini'].replace('-', 1) / (i * 10**6)
    df1[str(i) + ' mil'] = df1[str(i) + ' mil'].apply('{:,.1%}'.format)

df1['tmp'] = df1['300 mil'].str.replace('%', '').astype(float)
df1 = df1.sort_values('tmp').drop('tmp', axis=1).reset_index(drop=True)
df1

,ticker,name,crncy,risk,value/unit_JPY,mini,size_mini,200 mil,300 mil,500 mil,1000 mil
0,RTY,E-Mini Russ 2000 Sep24,USD,US,"15,772,000",HWR,0.1,0.8%,0.5%,0.3%,0.2%
1,XU,FTSE CHINA A50 Aug24,USD,CN,"1,717,000",-,-,0.9%,0.6%,0.3%,0.2%
2,TP,TOPIX INDX FUTR Sep24,JPY,JP,"26,745,000",TMI,0.1,1.3%,0.9%,0.5%,0.3%
3,HI,HANG SENG IDX FUT Aug24,HKD,HK,"16,367,000",HU,0.2,1.6%,1.1%,0.7%,0.3%
4,ES,S&P500 EMINI FUT Sep24,USD,US,"40,837,000",HWA,0.1,2.0%,1.4%,0.8%,0.4%
5,SW,MINI FTSE/MIB FUT Sep24,EUR,IT,"5,414,000",-,-,2.7%,1.8%,1.1%,0.5%
6,NQ,NASDAQ 100 E-MINI Sep24,USD,US,"57,233,000",HWB,0.1,2.9%,1.9%,1.1%,0.6%
7,DM,DJIA MINI e-CBOT Sep24,USD,US,"29,785,000",HWI,0.2,3.0%,2.0%,1.2%,0.6%
8,JGS,IFSC NIFTY 50 FUT Aug24,USD,IN,"7,237,000",-,-,3.6%,2.4%,1.4%,0.7%
9,VG,EURO STOXX 50 Sep24,EUR,EU,"7,925,000",-,-,4.0%,2.6%,1.6%,0.8%


In [224]:
def BDP(L,YK):
    T = [L[i] + 'A ' + YK[i] for i in range(len(L))]
    T1 = [L[i] + '1 ' + YK[i] for i in range(len(L))]
    
    BDP = blp.bdp(tickers=T, flds=['name','currency','country_iso','exch_code','fut_ctd_isin']).loc[T,:]
    isin = BDP.loc[T,['fut_ctd_isin']].dropna(how='all')
    isin = [i for i in isin['fut_ctd_isin']]

    tmp0 = ['year','dur','yld']
    tmp1 = ['mty_years','dur_adj_bid','yld_ytm_bid']


    BDP1 = blp.bdp(tickers = [i+' Corp' for i in isin], flds=tmp1)
    
    for i in isin:
        for j in range(len(tmp0)):
            BDP.loc[BDP['fut_ctd_isin']==i,tmp0[j]] = BDP1.loc[i+' Corp',tmp1[j]]

    BDP.insert(0,'ticker', L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=['contract_value','volume_avg_20d','volatility_20d']).loc[T1,:]
    BDP1.columns = ['contract_value','volume','vola']
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)
    

    BDP = BDP.rename(columns={'currency':'crncy','country_iso':'exch','exch_code':'exch_code','undl_spot_ticker':'undl'})
    
    for j in ['contract_value','volume']:
        BDP[j] = BDP[j].apply('{:,.0f}'.format)
        
    for j in ['year','dur','yld','vola']:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(blp.bdh(T1[i], 'px_last', '19991201', '20210101', Per='Q').reset_index().iloc[0,0]).strftime('%Y%m')]
    BDP['initinal_t/d'] = tmp
    BDP = BDP.replace('199912', '-')
    
    return BDP
    
L = ['TU','FV','TY','US','WN','DU','OE','RX','UB','OAT','IK','G ','CN','XM','JB']
# L = ['TU','FV']
YK = ['Comdty'] * len(L)

print('Bond Futures')
BDP = BDP(L,YK)
print(BDP.shape)
BDP

Bond Futures
(15, 13)


,ticker,name,crncy,exch,exch_code,fut_ctd_isin,year,dur,yld,contract_value,volume,vola,initinal_t/d
0,TU,US 2YR NOTE (CBT) Jun24,USD,US,CBT,US91282CKH33,1.84,1.73,4.97,"202,969","875,292",1.43,-
1,FV,US 5YR NOTE (CBT) Jun24,USD,US,CBT,US91282CHX20,4.26,3.80,4.58,"105,492","1,539,551",3.29,-
2,TY,US 10YR NOTE (CBT)Jun24,USD,US,CBT,US91282CKC46,6.75,5.75,4.49,"108,766","2,032,596",4.61,-
3,US,US LONG BOND(CBT) Jun24,USD,US,CBT,US912810QC53,15.21,10.79,4.53,"116,906","446,094",7.57,-
4,WN,US ULTRA BOND CBT Jun24,USD,US,CBT,US912810TV08,29.46,15.95,4.57,"123,844","374,767",9.68,201003
5,DU,EURO-SCHATZ FUT Jun24,EUR,DE,EUX,DE000BU22049,1.80,1.72,3.11,"105,030","542,898",1.23,-
6,OE,EURO-BOBL FUTURE Jun24,EUR,DE,EUX,DE000BU25026,4.87,4.53,2.61,"116,350","652,564",3.77,-
7,RX,EURO-BUND FUTURE Jun24,EUR,DE,EUX,DE000BU2Z007,8.71,7.74,2.51,"130,270","824,752",6.21,-
8,UB,EURO-BUXL 30Y BND Jun24,EUR,DE,EUX,DE0001102432,24.21,19.61,2.69,"128,940","112,507",13.61,-
9,OAT,Euro-OAT Future Jun24,EUR,DE,EUX,FR001400H7V7,8.99,7.78,2.98,"125,750","198,956",6.22,201206


In [226]:
def BDP(L,YK):
    T = [L[i] + 'A ' + YK[i] for i in range(len(L))]
    T1 = [L[i] + '1 ' + YK[i] for i in range(len(L))]

    BDP = blp.bdp(tickers=T, flds=['name','currency','country_iso','exch_code','undl_spot_ticker']).loc[T,:]
    BDP.insert(0,'ticker', L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=['contract_value','volume_avg_20d','volatility_20d']).loc[T1,:]
    BDP1.columns = ['contract_value','volume','vola']
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)

    BDP = BDP.rename(columns={'currency':'crncy','country_iso':'exch','exch_code':'exch_code','undl_spot_ticker':'undl'})
    
    tmp = ['contract_value','volume']
    for j in tmp:
        BDP[j] = BDP[j].apply('{:,.0f}'.format)
        
    for j in ['vola']:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(blp.bdh(T1[i], 'px_last', '19991201', '20220101', Per='Q' ).reset_index().iloc[0,0]).strftime('%Y%m')]
    BDP['initinal_t/d'] = tmp
    BDP = BDP.replace('199912', '-')
    
    return BDP
    
L= ['CL','CO','GC']
YK = ['Comdty'] * len(L)

print('Commodity Futures')
BDP = BDP(L,YK)
print(BDP.shape)
BDP

Commodity Futures
(3, 10)


,ticker,name,crncy,exch,exch_code,undl,contract_value,volume,vola,initinal_t/d
0,CL,WTI CRUDE FUTURE Jul24,USD,US,NYM,USCRWTIC,"78,920","285,079",15.67,-
1,CO,BRENT CRUDE FUTR Jul24,USD,GB,ICE,COY,"83,300","302,957",12.94,-
2,GC,GOLD 100 OZ FUTR Aug24,USD,US,CMX,NaN,"237,430","168,341",16.61,-


In [227]:
def BDP(L):
    T = ['CSBS' + i + ' TMUQ Curncy' for i in L]  #Mitsubishi TTM
    tmp = ['px_last','volatility_60d','rsi_30d']
    BDP = blp.bdp(tickers=T, flds=tmp).loc[T,:]
    BDP = BDP.loc[T,:]

    for i in tmp:
        BDP[i.lower()] = round(BDP[i.lower()].astype(float),2)
    BDP.insert(0, 'currency', L )
    BDP.insert(1, 'ticker', T)
    BDP = BDP.reset_index(drop=True)
    BDP = BDP.rename(columns={'volatility_60d':'vola'})

    return BDP

L = ['USD','EUR','GBP','CAD','AUD','NZD','CHF','NOK','SEK','HKD','CNY']

print('FX')
BDP = BDP(L)
print(BDP.shape)
BDP

FX
(11, 5)


,currency,ticker,px_last,vola,rsi_30d
0,USD,CSBSUSD TMUQ Curncy,156.94,8.11,59.81
1,EUR,CSBSEUR TMUQ Curncy,170.58,7.03,63.85
2,GBP,CSBSGBP TMUQ Curncy,200.49,7.75,65.44
3,CAD,CSBSCAD TMUQ Curncy,115.17,7.09,61.02
4,AUD,CSBSAUD TMUQ Curncy,104.46,8.52,64.84
5,NZD,CSBSNZD TMUQ Curncy,96.63,7.48,68.46
6,CHF,CSBSCHF TMUQ Curncy,171.91,6.83,55.75
7,NOK,CSBSNOK TMUQ Curncy,14.96,9.92,66.73
8,SEK,CSBSSEK TMUQ Curncy,14.80,8.47,63.20
9,HKD,CSBSHKD TMUQ Curncy,20.10,8.08,60.24


In [234]:
def BDP(L):
    T = [i + ' Index' for i in L]   
    BDP = blp.bdp(tickers=T, flds=['Security_name','long_comp_name','Currency','volatility_260d']).loc[T,:]
    BDP.insert(0, 'ticker', T)
    BDP = BDP.reset_index(drop=True)
    BDP = BDP.rename(columns={'volatility_260d':'vola'})
    BDP['vola'] = round(BDP['vola'].astype(float),2)

    for i in range(len(BDP)):
        tmp = blp.bdh(BDP.loc[i,'ticker'], ['px_last','tot_return_index_gross_dvds'], '20000101', '20230301')
        
        BDP.loc[i,'PR'] = round(tmp.iloc[len(tmp)-1,0] / tmp.iloc[0,0] - 1,2) 
        BDP.loc[i,'TR'] = round(tmp.iloc[len(tmp)-1,1] / tmp.iloc[0,1] - 1,2) 

        if tmp.iloc[len(tmp)-1,1] - tmp.iloc[len(tmp)-1,0] == 0:
            BDP.loc[i,'PR=TR'] = '1'
        else:
            BDP.loc[i,'PR=TR'] = ''

        BDP.loc[i,'Initial_TD'] = tmp.index[0].strftime('%Y%m')
    BDP = BDP.replace('200001', '-')
    return BDP
# https://www.jpmorgan.com/insights/research/index-research
# https://www.jpmorgan.com/content/dam/jpm/cib/complex/content/markets/index-research/Global-Index-Research-Product-Guide-2022.pdf#

In [235]:
print('Japan Stocks')
L = ['TPX','TPXDDVD']
BDP(L)

Japan Stocks


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,TPX Index,Tokyo Stock Exchange Tokyo Sto,Tokyo Stock Exchange Tokyo Stock Price Index T...,JPY,14.75,0.16,0.75,,-
1,TPXDDVD Index,Topix Total Return Index JPY,Topix Total Return Index JPY,JPY,14.63,0.75,0.75,1,-


In [236]:
print('Developed Country Stocks')
L = ['MBKO','M0KO','MXKO','M2KO','MAKO','MSHJKOK','GJ124280','HJ124336']
BDP(L)

Developed Country Stocks


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,MBKO Index,MSCI Kokusai JPY Price Index,MSCI Kokusai JPY Price Index,JPY,12.38,1.77,1.77,1,-
1,M0KO Index,MSCI Kokusai Gross Total Retur,MSCI Kokusai Gross Total Return JPY Index,JPY,12.38,3.77,3.77,1,-
2,MXKO Index,MSCI KOKUSAI Index,MSCI KOKUSAI Index,USD,10.61,1.06,2.63,,-
3,M2KO Index,MSCI Kokusai Gross Total Retur,MSCI Kokusai Gross Total Return USD Index,USD,10.62,2.59,2.59,1,-
4,MAKO Index,MSCI Kokusai JPY Net Index,MSCI Kokusai JPY Net Index,JPY,12.38,3.28,3.28,1,-
5,MSHJKOK Index,MSCI Hedged Indices In JPY Kok,MSCI Hedged Indices In JPY Kokusai,JPY,NaN,0.44,0.44,1,-
6,GJ124280 Index,MSCI Kokusai 100% Hedged to JP,MSCI Kokusai 100% Hedged to JPY Index,JPY,10.11,1.50,1.50,1,-
7,HJ124336 Index,MSCI Kokusai 100% Hedged to JP,MSCI Kokusai 100% Hedged to JPY,JPY,10.09,-0.00,-0.00,1,202102


In [237]:
print('Emerging Markets Stocks')
L = ['GDUEEGF','NDUEEGF','HJ137561']
BDP(L)

Emerging Markets Stocks


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,GDUEEGF Index,MSCI Daily TR Gross EM USD,MSCI Daily TR Gross EM USD,USD,12.82,2.68,2.68,1,-
1,NDUEEGF Index,MSCI Emerging Net Total Return,MSCI Emerging Net Total Return USD Index,USD,12.82,2.41,2.41,1,-
2,HJ137561 Index,MSCI EM (Emerging Markets) 100,MSCI EM (Emerging Markets) 100% Hedged to JPY ...,JPY,10.88,1.31,1.31,1,-


In [238]:
print('Developed Country REIT')
L = ['SREITWJJ','SREITJWJ']
BDP(L)

Developed Country REIT


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,SREITWJJ Index,S&P Developed Ex Japan REIT JP,S&P Developed Ex Japan REIT JPY Index,JPY,16.58,2.24,2.24,1,-
1,SREITJWJ Index,S&P Developed Ex Japan REIT To,S&P Developed Ex Japan REIT Total Return JPY I...,JPY,16.61,9.12,9.12,1,-


In [239]:
print('Japan Bonds')
L = ['BPITTO01']
BDP(L)

Japan Bonds


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,BPITTO01 Index,BPI Total Index,BPI Total Index,JPY,3.34,0.37,0.37,1,-


In [240]:
print('Developed Country Bond')
L = ['SBWGNJYC','SBWGNJYU']
BDP(L)

Developed Country Bond


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,SBWGNJYC Index,FTSE Non-JPY WGBI Curr-Hedged,FTSE Non-JPY WGBI Curr-Hedged JPY,JPY,5.06,0.46,0.46,1,-
1,SBWGNJYU Index,FTSE Non-JPY WGBI JPY,FTSE Non-JPY WGBI JPY,JPY,7.84,2.15,2.15,1,-


In [241]:
print('Emerging Market Bond / USD Bond')
L = ['JPGCUJCP','JPGCHJCP','JPEIDIVR','JPGCCOMP']
# L = ['JPGCUJCP','JPGCHJCP','JPEIDIVR','JPEIPLUS','JPEMCOMP','GBIE1545']
tmp = BDP(L)
tmp

Emerging Market Bond / USD Bond


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,JPGCUJCP Index,unhedged Return JPY,J.P. Morgan EMBI Global Diversified unhedged R...,JPY,8.71,1.10,1.10,1,201210
1,JPGCHJCP Index,Hedged JPY,J.P. Morgan EMBI Global Diversified Hedged JPY,JPY,6.03,1.89,1.89,1,-
2,JPEIDIVR Index,JPMorgan Monthly EMBIs,J.P. Morgan EMBI Global Diversified Composite,USD,6.09,3.68,3.68,1,-
3,JPGCCOMP Index,Composite,J.P. Morgan EMBI Global Diversified Composite,USD,6.04,3.68,3.68,1,-


In [242]:
print('Emerging Market Bond / Local Currency Bond')
L = ['JGENDVUJ','JGENDVHJ','JGENGUJG','JGENGHJG','JGENVUJG','JGENVHJG']
tmp = BDP(L)
tmp['Remarks'] = ['GBI EM Div']*2+['GBI EM Global']*2+['GBI EM Global Div']*2
tmp

Emerging Market Bond / Local Currency Bond


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD,Remarks
0,JGENDVUJ Index,Divers,J.P. Morgan GBI-EM Diversified JPY Unhedged,JPY,8.60,1.73,1.73,1,200212,GBI EM Div
1,JGENDVHJ Index,Divers,J.P. Morgan GBI-EM Diversified JPY Hedged,JPY,2.54,0.27,0.27,1,200301,GBI EM Div
2,JGENGUJG Index,JPY,J.P. Morgan GBI-EM Global Composite Unhedged JPY,JPY,7.78,2.57,2.57,1,200112,GBI EM Global
3,JGENGHJG Index,JPY,J.P. Morgan GBI-EM Global Composite Hedged JPY,JPY,1.32,0.53,0.53,1,200112,GBI EM Global
4,JGENVUJG Index,JPY,J.P. Morgan GBI-EM Global Diversified Composit...,JPY,8.43,2.04,2.04,1,200212,GBI EM Global Div
5,JGENVHJG Index,JPY,J.P. Morgan GBI-EM Global Diversified Composit...,JPY,2.29,0.39,0.39,1,200212,GBI EM Global Div
